# Laboratorio 4 | Machine Learning
* Marco Ferraro | B82957
* Roy Padilla | B85854

In [ ]:
import pandas as pd # procesamiento de dataset
import math # uso de función exp
import numpy as np # uso de funciones y generación de arreglos
import math # operación de raíz cuadrada

In [ ]:
target_value_titanic = ['Survived_0', 'Survived_1']
df_titanic = pd.read_csv('titanic.csv')
df_titanic = df_titanic.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin']).dropna()

# Se pasan a one-hot encoding las columnas 'Pclass','Sex' y 'Survived'
df_titanic = pd.get_dummies(df_titanic,columns=['Embarked', 'Pclass','Sex', 'Survived'])
y = df_titanic.loc[:, target_value_titanic]


df_titanic = df_titanic.drop(columns = target_value_titanic)

df_titanic.shape

(1043, 12)

1. Función `sigmoid(x)` que recibe un número `x` y retorna el resultado de la función sigmoide para dicho `x:   1/(1 + e^(-x))`


In [ ]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

2. Función d_sigmoid(y) que recibe un número y (tal que y = sigmoid(x)) y retorna el resultado de la derivada de la función sigmoide para el y dado: y*(1-y)


In [ ]:
def d_sigmoid(y):
  return y * (1 - y)

3. Función `tanh(x)` que recibe un número x y retorna el resultado de la función tangente hiperbólico para dicho `x: (e^x - e^(-x))/(e^x + e^(-x))`

In [ ]:
def tanh(x):
  return (math.exp(x) - math.exp(-x))/(math.exp(x) + math.exp(-x))

4. Función `d_tanh(y)` que recibe un número y (tal que `y = tanh(x)`) y retorna el resultado de la derivada de la función tangente hiperbólico para el `y` dado: `1 - y^2`

In [ ]:
def d_tanh(y):
  return 1-y**2

5. Función `relu(x)` que recibe un número x y retorna el resultado de la función lineal rectificada para dicho `x: (x if x > 0 else 0)`

In [ ]:
def relu(x):
  return x if x > 0 else 0

6. Función `d_relu(y)` que recibe un número `y` (tal que `y = relu(x)`) y retorna el resultado de la derivada de la función lineal rectificada para el y dado: `1 if y >0 else 0`

In [ ]:
def d_relu(y):
  return 1 if y > 0 else 0

7. `Función lrelu(x)` que recibe un número x y retorna el resultado de la función lineal rectificada con fuga para dicho x: (x if x>0 else 0.01x)

In [ ]:
def lrelu(x):
  return x if x>0 else 0.01*x

8. Función `d_lrelu(y)` que recibe un número y (tal que y = lrelu(x)) y retorna el resultado de la derivada de la función lineal rectificada con fuga para el y dado: 1 if y>0 else 0.01

In [ ]:
def d_lrelu(y):
  return 1 if y>0 else 0.01

`Clase DenseNN`


10. `Método predict(self, x)` que recibe una matriz de datos x de tamaño nxm, donde m coincide con el primer valor de layers al construir la red neuronal. Este método ejecuta el forward propagation aplicando la multiplicación con la matrices de pesos y las funciones de activación de cada capa. Finalmente, retorna una matriz de datos nxp, donde p coincide con el último de layers y corresponde a los valores predichos por la red neuronal para los casos x.

In [ ]:
class DenseNN:
    def __init__(self, layers, activation, seed = 0):
      np.random.seed(seed)

      self.x_count = layers[0]
      self.y_count = layers[-1]
      self.activation = activation
      self.epoch = 0
      self.momentum = 0
      self.lr = 0
      self.decay = 0

      self.weight_matrixs = []
      self.delta_weight_matrix = []
      
      self.net_values = []
      self.activation_values = []
      self.layers_errors = []

      # inicialización de la matriz de pesos y bias
      for index in range(len(layers[1:])):
        fan_in = layers[index] # capa anterior (cantidad de columnas)
        fan_out = 0 if len(layers[1:-1]) < index + 1 else layers[index+2] # capa siguiente
        self.weight_matrixs.append(np.random.normal(loc = 0, scale=2.0/math.sqrt(fan_in + fan_out), size = (fan_in + 1, layers[index + 1])))

    def predict(self, x = pd.DataFrame):
      x = x.assign(bias = np.ones(x.shape[0]))
      predictions = np.array([])
      for index, row in x.iterrows():
        # llamada al forward propagation para la fila
        predictions = np.append(predictions, self._fowardPropagation(row))
      return predictions 

    def train(self, lr=0.05, momentum=0, decay=0):
      self.epoch = 0
      # REVISAR
      self.delta_weight_mat rix = [[[0]*len(self.weight_matrixs[0][0])]*len(self.weight_matrixs[0])]*len(self.weight_matrixs)
      self.lr = lr
      self.momentum = momentum
      self.decay = decay

    def _fowardPropagation(self, row):
      # limpieza de los valores netos, activación y error
      self.activation_values = []
      self.net_values = []
      self.layers_errors = []
      # inicialización de los valores para la primer capa oculta
      net_k = np.matmul(np.transpose(row.to_numpy()), self.weight_matrixs[0])
      o_k = self._getActivationValue(0, net_k)
      
      self.net_values.append(net_k)
      
      # recorrido desde la segunda capa oculta hasta la salida
      for weight_index in range(len(self.weight_matrixs[1:])):
        # aplicación de la multiplicación: x*w
        net_k = np.matmul(np.transpose(np.append(o_k, 1)), self.weight_matrixs[weight_index + 1])
        # Aplicación de la función de activación en cada uno de los resultados
        o_k = self._getActivationValue(weight_index + 1, net_k)
        self.net_values.append(net_k)
        self.activation_values.append(o_k)
      return o_k

    def backPropagation(self, x, y):
      x = x.assign(bias = np.ones(x.shape[0]))
      for index, row in x.iterrows():
        y_predict = self._fowardPropagation(row)
        y_true = y.loc[index]
        dj = []
        for layer_index in reversed(range(len(self.weight_matrixs))):
          if layer_index == len(self.weight_matrixs) - 1:
            dj = 2 * (y_predict - y_true) * self._getDerivativeActivationValues(layer_index, self.net_values[layer_index])
          else:
            
            dj = (self.weight_matrixs[layer_index+1] @ dj) * np.append(self._getDerivativeActivationValues(layer_index, self.net_values[layer_index]), 1)

            # for i in range(len(dj)):
            #   sum += self.weight_matrixs[layer_index][i] * self._getDeltaLastLayer()
            # dj = sum * self._getDerivativeActivationValues(layer_index, self.net_values[layer_index])
            
    def step(self):
      # apply backpropagation
      self.epoch += 1

    def _getDeltaLastLayer(self):
      
      return 0

    def _getDeltaHidingLayer(self):
      return 0

    def _getActivationValue(self, layer_number, values):
      return_values = []
      for value in np.nditer(values):
        return_values.append(self._applyActivation(layer_number, value))
      return np.array(return_values)

    def _getDerivativeActivationValues(self, layer_number, values):
      return_values = []
      for value in np.nditer(values):
        return_values.append(self._applyActivationDerivative(layer_number, value))
      return np.array(return_values)

    def _applyActivation(self, layer_number, value):
      if self.activation[layer_number] == 's': # aplicar función sigmoide
        return sigmoid(value)
      elif self.activation[layer_number] == 't': # aplicar función tangente hiperbólico
        return tanh(value)
      elif self.activation[layer_number] == 'r': # aplicar función relu
        return relu(value)
      elif self.activation[layer_number] == 'l': # aplicar función leaky relu
        return lrelu(value)
      return 0

    def _applyActivationDerivative(self, layer_number, value):
      if self.activation[layer_number] == 's': # aplicar función sigmoide
        return d_sigmoid(value)
      elif self.activation[layer_number] == 't': # aplicar función tangente hiperbólico
        return d_tanh(value)
      elif self.activation[layer_number] == 'r': # aplicar función relu
        return d_relu(value)
      elif self.activation[layer_number] == 'l': # aplicar función leaky relu
        return d_lrelu(value)
      return 0


In [ ]:
# dismunución del dataset para pruebas
df_titanic = df_titanic[:4]
y = y[:4]

print(df_titanic.shape)
neural_net = DenseNN([12, 10, 5, 2],['r', 's', 's'], 0)
neural_net.predict(df_titanic)

(4, 12)


array([0.50849989, 0.46811816, 0.53291155, 0.49838054, 0.50258385,
       0.47866953, 0.53295475, 0.49841047])

In [ ]:
neural_net.backPropagation(df_titanic, y)
# neural_net.train()